In [ ]:
import numpy as np 
from functions import function
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import trange

In [ ]:
# Plot the dataset
def plot_data(ax, X, Y):
    plt.axis('off')
    ax.scatter(X[:, 0], X[:, 1], s=1, c=Y, cmap='bone')
    
# plot the decision boundary of our classifier
def plot_decision_boundary(ax, x_min, x_max, y_min, y_max, X, Y, classifier):
    # forward pass on the grid, then convert to numpy for plotting
    # Define the grid on which we will evaluate our classifier
    xx, yy = np.meshgrid(np.arange(x_min, x_max, .1),
                        np.arange(y_min, y_max, .1))

    to_forward = np.array(list(zip(xx.ravel(), yy.ravel())))
    Z = []
    for x in to_forward:
        Z.append(classifier.forward(x, sample = True))
    Z = np.array(Z)
    Z = Z.reshape(xx.shape)
    
    # plot contour lines of the values of our classifier on the grid
    ax.contourf(xx, yy, Z>0.5, cmap='Blues')
    
    # then plot the dataset
    plot_data(ax, X,Y)




In [ ]:
X, Y = make_moons(n_samples=2000, noise=0.1)
x_min, x_max = -1.5, 2.5
y_min, y_max = -1, 1.5
fig, ax = plt.subplots(1, 1, facecolor='#4B6EA9')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plot_data(ax, X, Y)
plt.show()



In [ ]:
class ReLU(object):
    def forward(self, x, sample):
        self.x = x
        # print('ReLU shape',self.x.shape)
        return np.maximum(0, x)
        
    def backward(self, grad_output):
        return grad_output * (self.x > 0)
    
    def step(self, learning_rate):
        pass

class Sigmoid(object):
    def forward(self, x, sample):
        self.output = 1 / (1 + np.exp(-x))
        # print('Sigmoid shape',self.output.shape)
        return self.output

    def backward(self, grad_output):
        return grad_output * self.output * (1 - self.output)
    
    def step(self, learning_rate):
        pass



In [ ]:
class Linear(object):
    def __init__(self, n_input, n_output, sigma_w = None, sigma_b = None):
        self.weights = np.random.randn(n_output, n_input)
        self.bias = np.random.randn(n_output,1)
        
        if sigma_w is not None:
            self.sigma_w = sigma_w
        else:
            self.sigma_w = np.eye(n_output*n_input)*1e-3
        
        if sigma_b is not None:
            self.sigma_b = sigma_b
        else:
            self.sigma_b = np.eye(n_output)*1e-3

    def forward(self, x, sample = True):
        self.x = x.reshape(-1,1)
        if sample:
            self.weights =  np.random.multivariate_normal(mean = self.weights.flatten(), cov = self.sigma_w).reshape(self.weights.shape)
            self.bias =  np.random.multivariate_normal(mean = self.bias.flatten(), cov = self.sigma_b).reshape(self.bias.shape)
        return self.weights@self.x + self.bias
    
    def backward(self, grad_output):
        self.grad_weights = grad_output@self.x.T
        self.grad_bias = np.sum(grad_output, axis = 1).reshape(self.bias.shape)
        return self.weights.T@grad_output
    
    def step(self, learning_rate):
        self.weights -= (np.linalg.pinv(self.sigma_w)@self.grad_weights.flatten()).reshape(self.weights.shape) * learning_rate
        self.bias -= (np.linalg.pinv(self.sigma_b)@self.grad_bias.flatten()).reshape(self.bias.shape) * learning_rate



In [ ]:
class Sequential(object):
    def __init__(self, layers):
        self.layers = layers

    def forward(self, x, sample):
        for layer in self.layers:
            x = layer.forward(x, sample)
        return x
    
    def compute_loss(self, out, label):
        BCE = -np.mean(label*np.log(out+1e-8)+(1-label)*np.log(1-out+1e-8))
        self.grad_output = (-(label/(out+1e-10) - (1 - label) / (1 - out+1e-10))).reshape(-1,1)
        return BCE

    def backward(self):
        grad_output = self.grad_output
        # print('Begin backward ')
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output)


    def step(self, learning_rate):
        for layer in self.layers:
            layer.step(learning_rate)

In [ ]:
losses = []
batch_size = 100
learning_rate = 1e-5


h=50
net = Sequential([Linear(2, h), ReLU(), Linear(h, 1), Sigmoid()])

for it in trange(5000):
    
    # pick a random example id
    batch_indices = np.random.choice(X.shape[0], batch_size, replace=False)

    # select the corresponding example and label
    loss = 0
    for index in batch_indices:
        example = X[index,:]
        label = Y[index]
        if it%11==0:
            out = net.forward(example, sample = True)
        else:
            out = net.forward(example, sample = False)
        loss += net.compute_loss(out, label)

    # backward pass    
    losses.append(loss)
 
    # backward pass
    net.backward()
    
    # gradient step
    net.step(learning_rate)

    # draw the current decision boundary every 250 examples seen
    if it % 250 == 0 : 
        fig, ax = plt.subplots(1, 1, facecolor='#4B6EA9')
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        plot_decision_boundary(ax, x_min, x_max, y_min, y_max, X, Y, net)
        plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, facecolor='#4B6EA9')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plot_decision_boundary(ax, x_min, x_max, y_min, y_max, X, Y, net)
plt.show()

plt.figure()
plt.plot(losses)
plt.title('Evolution of the loss during learning')
plt.xlabel('Number of iterations')
plt.ylabel('BCE')
plt.show()